<a href="https://colab.research.google.com/github/ParthSalat47/NLP-in-Neo4j/blob/master/polyglot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mounting Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Entity extraction with polyglot

Use the Polyglot Python library to enrich the tweet graph with extracted entities

In [ ]:
!pip install polyglot

     |████████████████████████████████| 126 kB 2.1 MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52578 sha256=40589f259ec32507a9b2f5919d35b66de012717e550842d54149fc036c81fc14
  Stored in directory: /root/.cache/pip/wheels/09/bc/67/75c9de8e9726460bc0b101ad225ad025cb8ce9e0759beb9d52
Successfully built polyglot


In [ ]:
#!polyglot download embeddings2.en ner2.en

Traceback (most recent call last):
  File "/usr/local/bin/polyglot", line 5, in <module>
    from polyglot.__main__ import main
  File "/usr/local/lib/python3.7/dist-packages/polyglot/__main__.py", line 16, in <module>
    from icu import Locale
ModuleNotFoundError: No module named 'icu'


In [ ]:
!pip install neo4j
from neo4j import GraphDatabase
import json
import pprint
#from polyglot.text import Text

In [ ]:

from neo4j import GraphDatabase
driver = GraphDatabase.driver("neo4j+s://1997272c.databases.neo4j.io:7687", auth=("neo4j", "DeTjqssc064er5wkA981vUaTyc09Jhz68m8WZ5WZq_k"))  
#neo4j://127.0.0.1:7687
#neo4j://127.0.0.1:7474


with driver.session() as session:
    results = session.run("MATCH (t:Tweet) RETURN t.text AS text, t.tweet_id AS tweet_id")

tweetObjArr = []

for r in results:
    tweetObj = {}
    tweetObj['id'] = r['tweet_id']
    tweetObj['text'] = r['text']
    tweetObjArr.append(tweetObj)



             


In [ ]:
len(tweetObjArr)

0

In [ ]:
entityArr = []


for t in tweetObjArr:
    try:
        parsedTweet = {}
        parsedTweet['id'] = t['id']
        parsedTweet['text'] = t['text']
        blob = Text(t['text'])
        entities = blob.entities
        parsedTweet['entities'] = []
        for e in entities:
            eobj = {}
            eobj['tag'] = e.tag
            eobj['entity'] = e
            parsedTweet['entities'].append(eobj)
        if len(parsedTweet['entities']) > 0:
            entityArr.append(parsedTweet)
    except:
        pass

In [ ]:
import json
with open('parsed_tweets_scraped.json', 'w') as f:
    json.dump(entityArr, f, ensure_ascii=False, sort_keys=True, indent=4)

In [ ]:
len(entityArr)

In [ ]:
entityArr[5]

{'entities': [{'entity': I-PER(['Hillary', 'Clinton']), 'tag': 'I-PER'}],
 'id': 773585101489922048,
 'text': '@realDonaldTrump "Hillary Clinton has zero record to run on - unless you call corruption positive.." - @IngrahamAngle'}

# Import into Neo4j

In [ ]:
with open("parsed_tweets_scraped.json") as f:
    parsed_tweets = json.load(f)

In [ ]:
len(parsed_tweets)

589

In [ ]:
pprint.pprint(parsed_tweets[4])

{'entities': [{'entity': ['Hillary'], 'tag': 'I-PER'}],
 'id': '588771323289030657',
 'text': 'Emails investigation made Hillary a dubious candidate #DemsWontPass'}


In [ ]:
driver = GraphDatabase.driver("bolt://localhost:7687")

with driver.session() as session:
    session.run('CREATE CONSTRAINT ON (p:Person) ASSERT p.name IS UNIQUE;')
    session.run('CREATE CONSTRAINT ON (l:Location) ASSERT l.name IS UNIQUE;')
    session.run('CREATE CONSTRAINT ON (o:Organization) ASSERT o.name IS UNIQUE;')

In [ ]:
entity_import_query = '''
WITH $parsedTweets AS parsedTweets
UNWIND parsedTweets AS parsedTweet
MATCH (t:Tweet) WHERE t.tweet_id = parsedTweet.id


FOREACH(entity IN parsedTweet.entities |
    // Person
    FOREACH(_ IN CASE WHEN entity.tag = 'I-PER' THEN [1] ELSE [] END | 
        MERGE (p:Person {name: reduce(s = "", x IN entity.entity | s + x + " ")}) //FIXME: trailing space
        MERGE (p)<-[:CONTAINS_ENTITY]-(t)
    )
    
    // Organization
    FOREACH(_ IN CASE WHEN entity.tag = 'I-ORG' THEN [1] ELSE [] END | 
        MERGE (o:Organization {name: reduce(s = "", x IN entity.entity | s + x + " ")}) //FIXME: trailing space
        MERGE (o)<-[:CONTAINS_ENTITY]-(t)
    )
    
    // Location
    FOREACH(_ IN CASE WHEN entity.tag = 'I-LOC' THEN [1] ELSE [] END | 
        MERGE (l:Location {name: reduce(s = "", x IN entity.entity | s + x + " ")}) // FIXME: trailing space
        MERGE (l)<-[:CONTAINS_ENTITY]-(t)
    )
)

'''

In [ ]:
with driver.session() as session:
    session.run(entity_import_query, parsedTweets=parsed_tweets)